In [3]:
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import pysam
import os
import ast
import numpy as np
import json
import collections

In [5]:
hg38Merged = pd.read_csv('/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/old_wScott/hg38/hg38_MergedCallSet_PASSED_06-18-2024_Clean.csv').set_index("Unnamed: 0")
print(len(hg38Merged))

14210


In [6]:
#Read In the Merged hg38 Set and Filter out Arthurs removed calls
t2tMerged = pd.read_csv('/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/old_wScott/t2t/t2t_MergedCallSet_PASSED_06-18-2024_Clean.csv').set_index("Unnamed: 0")
print(len(t2tMerged))

14404


In [7]:
scottshg38Calls=[x for x in hg38Merged[(hg38Merged['Caller_Count']==1) & (hg38Merged['Devine_Lab']==1)].index]
scottsT2TCalls=[x for x in t2tMerged[(t2tMerged['Caller_Count']==1) & (t2tMerged['Devine_Lab']==1)].index]

In [8]:
print(len(scottshg38Calls))
print(len(scottsT2TCalls))

219
196


In [9]:
hg38Cleaned = hg38Merged.loc[[x for x in hg38Merged.index if x not in scottshg38Calls]].drop(columns=['Devine_Lab', 'Devine_Lab_INFO']).copy()
t2tCleaned = t2tMerged.loc[[x for x in t2tMerged.index if x not in scottsT2TCalls]].drop(columns=['Devine_Lab', 'Devine_Lab_INFO']).copy()

In [10]:
hg38Cleaned['Caller_Count'] = [int(x)+int(y) for x,y in zip(hg38Cleaned['Konkel_Lab'],hg38Cleaned['PALMER'])]
t2tCleaned['Caller_Count'] = [int(x)+int(y) for x,y in zip(t2tCleaned['Konkel_Lab'],t2tCleaned['PALMER'])]

In [11]:
#hg38Cleaned.to_csv('/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/hg38_allCalls_07-24-2024.csv')
#t2tCleaned.to_csv('/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/hs1_allCalls_07-24-2024.csv')

In [12]:
arthurhg38 = hg38Cleaned[(hg38Cleaned['Caller_Count']==1) & (hg38Cleaned['PALMER']==1)].copy()
arthurhs1 = t2tCleaned[(t2tCleaned['Caller_Count']==1) & (t2tCleaned['PALMER']==1)].copy()

In [13]:
konkelhg38 = hg38Cleaned[(hg38Cleaned['Caller_Count']==1) & (hg38Cleaned['Konkel_Lab']==1)].copy()
konkelhs1 = t2tCleaned[(t2tCleaned['Caller_Count']==1) & (t2tCleaned['Konkel_Lab']==1)].copy()

In [42]:
#directory = '/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/ArthurSequences/hg38/'
#with open(directory+'arthurhg38_Sequences.fasta','a+') as file:
#    for row in arthurhg38.index:
#        file.write(">"+str(row)+'\n')
#        file.write(str(arthurhg38.at[row,'ALT'])+"\n")
#file.close()

#directory = '/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/ArthurSequences/hs1/'
#with open(directory+'arthurhs1_Sequences.fasta','a+') as file:
#    for row in arthurhs1.index:
#        file.write(">"+str(row)+'\n')
#        file.write(str(arthurhs1.at[row,'ALT'])+"\n")
#file.close()

In [43]:
#directory = '/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/KonkelSequences/hg38/'
#with open(directory+'konkelhg38_Sequences.fasta','a+') as file:
#    for row in konkelhg38.index:
#        file.write(">"+str(row)+'\n')
#        file.write(str(konkelhg38.at[row,'ALT'])+"\n")
#file.close()

#directory = '/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/KonkelSequences/hs1/'
#with open(directory+'konkelhs1_Sequences.fasta','a+') as file:
#    for row in konkelhs1.index:
#        file.write(">"+str(row)+'\n')
#        file.write(str(konkelhs1.at[row,'ALT'])+"\n")
#file.close()

## Filter out the HERVs that need to be removed manually 

In [9]:
hg38Cleaned = pd.read_csv('/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/hg38_hgsvc3_finalManuscript.csv').set_index("ID")
t2tCleaned=pd.read_csv('/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/hs1_hgsvc3_finalManuscript.csv').set_index("ID")

In [20]:
hg38bad = ['MEI_14010-chr8-145027990']
t2tbad = ['MEI_13634-chr8-11959656', 'MEI_13636-chr8-12500180']

In [23]:
hg382 = hg38Cleaned.loc[[x for x in hg38Cleaned.index if x not in hg38bad]].copy()

In [24]:
t2t2 = t2tCleaned.loc[[x for x in t2tCleaned.index if x not in t2tbad]].copy()

In [26]:
print(len(hg382))
print(len(t2t2))

13082
13298


In [25]:
#hg382.to_csv("/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/GRCh38_hgsvc3_PublicationSet.csv")
#t2t2.to_csv("/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/t2tCHM13_hgsvc3_PublicationSet.csv")

## Deduplicate PAV "Calls

In [156]:
hg38 = pd.read_csv("/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/GRCh38_hgsvc3_PublicationSet.csv").set_index("ID")
t2t2=pd.read_csv("/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/t2tCHM13_hgsvc3_PublicationSet.csv").set_index("ID")

In [157]:
unique = []
duplicated=[]
allCalls = [str(x)+"_"+str(y) for x,y in zip(hg38['CHROM'], hg38['POS'])]
dupsDict= collections.Counter(allCalls)
for call in hg38.index:
    chromPos = str(hg38.at[call,'CHROM'])+"_"+str(hg38.at[call,'POS'])
    if dupsDict[chromPos]==1:
        unique.append(call)
    else:
        duplicated.append(call)
uniquehg38 = hg38.loc[unique].copy()
tempDupDF = hg38.loc[duplicated].copy()
tempDupDF['chromLoc']=[str(x)+"_"+str(y) for x,y in zip(tempDupDF['CHROM'], tempDupDF['POS'])]
sampleDict={x:'temp' for x in ['HG00096', 'HG00171', 'HG00268', 'HG00358', 'HG00512', 'HG00513',
       'HG00514', 'HG00731', 'HG00732', 'HG00733', 'HG00864', 'HG01114',
       'HG01352', 'HG01457', 'HG01505', 'HG01573', 'HG01596', 'HG01890',
       'HG02011', 'HG02018', 'HG02059', 'HG02106', 'HG02282', 'HG02492',
       'HG02554', 'HG02587', 'HG02666', 'HG02769', 'HG02818', 'HG02953',
       'HG03009', 'HG03065', 'HG03248', 'HG03371', 'HG03452', 'HG03456',
       'HG03520', 'HG03683', 'HG03732', 'HG03807', 'HG04036', 'HG04217',
       'NA12329', 'NA18534', 'NA18939', 'NA18989', 'NA19036', 'NA19129',
       'NA19238', 'NA19239', 'NA19240', 'NA19317', 'NA19331', 'NA19347',
       'NA19384', 'NA19434', 'NA19650', 'NA19705', 'NA19836', 'NA19983',
       'NA20355', 'NA20509', 'NA20847', 'NA21487', 'NA24385']}

dedupDict={}
additionalCops=[]
for dupPosition in set(tempDupDF['chromLoc']):
        
    dedupDict[dupPosition]={'Chosen':'temp', 'Duplications':[]}
    testDF = tempDupDF[tempDupDF['chromLoc']==dupPosition].copy()
    
    #See if both callers are having hits
    arthurCount=0
    limeCount=0
    PALMERrow='temp'
    PALMERsinglerow='temp'
    for row in testDF.index:
        
        if int(testDF.at[row,'L1ME-AID_Lab'])==1 and int(testDF.at[row,'PALMER'])==1:
            limeCount+=1
            arthurCount+=1
            PALMERrow=row
            
        elif int(testDF.at[row,'L1ME-AID_Lab'])==0 and int(testDF.at[row,'PALMER'])==1:
            arthurCount+=1
            PALMERsinglerow=row
        
        elif int(testDF.at[row,'L1ME-AID_Lab'])==1 and int(testDF.at[row,'PALMER'])==0:
            limeCount+=1
        
        else:
            print('HUH')
            
    #if PALMERrow !='temp' and PALMERsinglerow!='temp':
    #    print(dupPosition)
    #else:
    #    pass
    
    #If no hits were called by PALMER then worry about intra-PAV duplications
    if arthurCount == 0 and len(set(testDF['TE_Designation']))==1:
        
        
        largest='temp'
        largestLength=0
        for row in testDF.index:
            if int(row.split("-")[-1])>largestLength:
                largestLength = int(row.split("-")[-1])
                largest = row
            else:
                continue
                
        
        dedupDict[dupPosition]['Chosen']=largest
        dedupDict[dupPosition]['Duplications']=[y for y in testDF.index if y !=largest]
        
        for sample in sampleDict.keys():
            pos1='0'
            pos2='0'
            pos3='0'
            
            for geno in testDF[sample]:
                if str(geno.split("|")[0]) != '0':
                    pos1 =str(geno.split("|")[0])
                    
                if str(geno.split("|")[1]) != '0':
                    pos2 =str(geno.split("|")[1])
                    
                if str(geno.split("|")[2]) != '0':
                    pos3 =str(geno.split("|")[2])
                    
                    
            newString = str(pos1+"|"+pos2+"|"+pos3)
            tempDupDF.at[largest,sample]=newString
             
        
    #Else if hits were also called by PALMER
    elif arthurCount !=0 and len(set(testDF['TE_Designation']))==1:
        
        if PALMERrow !='temp':
            largest=PALMERrow
            largestLength=0
            dedupDict[dupPosition]['Chosen']=largest
            dedupDict[dupPosition]['Duplications']=[y for y in testDF.index if y !=largest]
            
            # Merge genotyping
            for sample in sampleDict.keys():
                pos1='0'
                pos2='0'
                pos3='0'

                for row in testDF.index:
                    if 'MEI' in str(row):
                        geno = str(testDF.at[row,sample])
                        if str(geno.split("|")[0]) != '0':
                            pos1 =str(geno.split("|")[0])

                        if str(geno.split("|")[1]) != '0':
                            pos2 =str(geno.split("|")[1])

                    else:
                        geno = str(testDF.at[row,sample])
                        if str(geno.split("|")[0]) != '0':
                            pos1 =str(geno.split("|")[0])

                        if str(geno.split("|")[1]) != '0':
                            pos2 =str(geno.split("|")[1])

                        if str(geno.split("|")[2]) != '0':
                            pos3 =str(geno.split("|")[2])


                newString = str(pos1+"|"+pos2+"|"+pos3)
                tempDupDF.at[largest,sample]=newString
            
            
        else:
            
            largest='temp'
            largestLength=0
            for row in testDF.index:
                if len(testDF.at[row,'ALT'])>largestLength:
                    largestLength = len(testDF.at[row,'ALT'])
                    largest = row
                else:
                    continue
                    
            if largest == PALMERsinglerow:
                reverseFlag=1
            else:
                reverseFlag=0


            dedupDict[dupPosition]['Chosen']=largest
            dedupDict[dupPosition]['Duplications']=[y for y in testDF.index if y !=largest]
            
            # Merge genotyping
            for sample in sampleDict.keys():
                pos1='0'
                pos2='0'
                pos3='0'

                for row in testDF.index:
                    if 'MEI' in str(row):
                        geno = str(testDF.at[row,sample])
                        if str(geno.split("|")[0]) != '0':
                            pos1 =str(geno.split("|")[0])

                        if str(geno.split("|")[1]) != '0':
                            pos2 =str(geno.split("|")[1])

                    else:
                        geno = str(testDF.at[row,sample])
                        if str(geno.split("|")[0]) != '0':
                            pos1 =str(geno.split("|")[0])

                        if str(geno.split("|")[1]) != '0':
                            pos2 =str(geno.split("|")[1])

                        if str(geno.split("|")[2]) != '0':
                            pos3 =str(geno.split("|")[2])


                newString = str(pos1+"|"+pos2+"|"+pos3)
                tempDupDF.at[largest,sample]=newString
            
            
            if reverseFlag == 0:
                tempDupDF.at[largest,'Caller_Count']=2
                tempDupDF.at[largest,'PALMER']=1
                tempDupDF.at[largest,'PALMER_INFO']=tempDupDF.at[PALMERsinglerow,'PALMER_INFO']
                tempDupDF.at[largest,'L1ME-AID_Lab']=1
                
            else:
                singleHit = dedupDict[dupPosition]['Duplications'][0]
                
                tempDupDF.at[largest,'Caller_Count']=2
                tempDupDF.at[largest,'PALMER']=1
                tempDupDF.at[largest,'L1ME-AID_Lab']=1
                tempDupDF.at[largest,'L1ME-AID_INFO']=tempDupDF.at[singleHit,'L1ME-AID_INFO']
        

        
    else:
        print(dupPosition)
        print(set(testDF['TE_Designation']))
        print("HUH3") 
        
        
        
        
tempDupDF['MergedCalls']='NONE'
for row in tempDupDF.index:
    for element in dedupDict.keys():
        if dedupDict[element]['Chosen'] ==  row:
            tempDupDF.at[row,'MergedCalls'] = dedupDict[element]['Duplications']
        else:
            continue
            
chosenRows=[]
for x in dedupDict.keys():
    for y in dedupDict[x]['Duplications']:
        chosenRows.append(y)
        
deduplicatedDF = tempDupDF.loc[[x for x in tempDupDF.index if x not in chosenRows and x not in additionalCops]].copy()
uniquehg38['chromLoc']='NONE'
uniquehg38['MergedCalls']='NONE'
concatList=[uniquehg38,deduplicatedDF]
putBackDF = pd.concat(concatList)
putBackDF.drop(columns=['chromLoc'], inplace=True)
putBackDF2 = putBackDF.reset_index().copy()
testRows=[]
for row in putBackDF2.index:
    originalID = str(putBackDF2.at[row,'ID'])
    if 'MEI' in originalID and putBackDF2.at[row,'MergedCalls']!='NONE':
        testRows.append(row)
        
        pavName = putBackDF2.at[row,'MergedCalls'][0]

        putBackDF2.at[row,'ID']=pavName
        putBackDF2.at[row,'ALT']=hg38.at[pavName,'ALT']

        putBackDF2.at[row,'MergedCalls']='temp'
        putBackDF2.at[row,'MergedCalls']=[originalID]


        
    else:
        continue
putBackDF2.set_index("ID", inplace=True)
finalhg38Demerged = putBackDF2.copy()

chr9_129033138
{'SINE/Alu', 'Retroposon/SVA'}
HUH3


In [158]:
unique = []
duplicated=[]
allCalls = [str(x)+"_"+str(y) for x,y in zip(t2t2['CHROM'], t2t2['POS'])]
dupsDict= collections.Counter(allCalls)
for call in t2t2.index:
    chromPos = str(t2t2.at[call,'CHROM'])+"_"+str(t2t2.at[call,'POS'])
    if dupsDict[chromPos]==1:
        unique.append(call)
    else:
        duplicated.append(call)
uniquet2t2 = t2t2.loc[unique].copy()
tempDupDF = t2t2.loc[duplicated].copy()
tempDupDF['chromLoc']=[str(x)+"_"+str(y) for x,y in zip(tempDupDF['CHROM'], tempDupDF['POS'])]
sampleDict={x:'temp' for x in ['HG00096', 'HG00171', 'HG00268', 'HG00358', 'HG00512', 'HG00513',
       'HG00514', 'HG00731', 'HG00732', 'HG00733', 'HG00864', 'HG01114',
       'HG01352', 'HG01457', 'HG01505', 'HG01573', 'HG01596', 'HG01890',
       'HG02011', 'HG02018', 'HG02059', 'HG02106', 'HG02282', 'HG02492',
       'HG02554', 'HG02587', 'HG02666', 'HG02769', 'HG02818', 'HG02953',
       'HG03009', 'HG03065', 'HG03248', 'HG03371', 'HG03452', 'HG03456',
       'HG03520', 'HG03683', 'HG03732', 'HG03807', 'HG04036', 'HG04217',
       'NA12329', 'NA18534', 'NA18939', 'NA18989', 'NA19036', 'NA19129',
       'NA19238', 'NA19239', 'NA19240', 'NA19317', 'NA19331', 'NA19347',
       'NA19384', 'NA19434', 'NA19650', 'NA19705', 'NA19836', 'NA19983',
       'NA20355', 'NA20509', 'NA20847', 'NA21487', 'NA24385']}
dedupDict={}
additionalCops=[]
for dupPosition in set(tempDupDF['chromLoc']):
        
    dedupDict[dupPosition]={'Chosen':'temp', 'Duplications':[]}
    testDF = tempDupDF[tempDupDF['chromLoc']==dupPosition].copy()
    
    #See if both callers are having hits
    arthurCount=0
    limeCount=0
    PALMERrow='temp'
    PALMERsinglerow='temp'
    for row in testDF.index:
        
        if int(testDF.at[row,'L1ME-AID_Lab'])==1 and int(testDF.at[row,'PALMER'])==1:
            limeCount+=1
            arthurCount+=1
            PALMERrow=row
            
        elif int(testDF.at[row,'L1ME-AID_Lab'])==0 and int(testDF.at[row,'PALMER'])==1:
            arthurCount+=1
            PALMERsinglerow=row
        
        elif int(testDF.at[row,'L1ME-AID_Lab'])==1 and int(testDF.at[row,'PALMER'])==0:
            limeCount+=1
        
        else:
            print('HUH')
            
    #if PALMERrow !='temp' and PALMERsinglerow!='temp':
    #    print(dupPosition)
    #else:
    #    pass
    
    #If no hits were called by PALMER then worry about intra-PAV duplications
    if arthurCount == 0 and len(set(testDF['TE_Designation']))==1:
        
        
        largest='temp'
        largestLength=0
        for row in testDF.index:
            if int(row.split("-")[-1])>largestLength:
                largestLength = int(row.split("-")[-1])
                largest = row
            else:
                continue
                
        
        dedupDict[dupPosition]['Chosen']=largest
        dedupDict[dupPosition]['Duplications']=[y for y in testDF.index if y !=largest]
        
        for sample in sampleDict.keys():
            pos1='0'
            pos2='0'
            pos3='0'
            
            for geno in testDF[sample]:
                if str(geno.split("|")[0]) != '0':
                    pos1 =str(geno.split("|")[0])
                    
                if str(geno.split("|")[1]) != '0':
                    pos2 =str(geno.split("|")[1])
                    
                if str(geno.split("|")[2]) != '0':
                    pos3 =str(geno.split("|")[2])
                    
                    
            newString = str(pos1+"|"+pos2+"|"+pos3)
            tempDupDF.at[largest,sample]=newString
             
        
    #Else if hits were also called by PALMER
    elif arthurCount !=0 and len(set(testDF['TE_Designation']))==1:
        
        if PALMERrow !='temp':
            largest=PALMERrow
            largestLength=0
            dedupDict[dupPosition]['Chosen']=largest
            dedupDict[dupPosition]['Duplications']=[y for y in testDF.index if y !=largest]
            
            # Merge genotyping
            for sample in sampleDict.keys():
                pos1='0'
                pos2='0'
                pos3='0'

                for row in testDF.index:
                    if 'MEI' in str(row):
                        geno = str(testDF.at[row,sample])
                        if str(geno.split("|")[0]) != '0':
                            pos1 =str(geno.split("|")[0])

                        if str(geno.split("|")[1]) != '0':
                            pos2 =str(geno.split("|")[1])

                    else:
                        geno = str(testDF.at[row,sample])
                        if str(geno.split("|")[0]) != '0':
                            pos1 =str(geno.split("|")[0])

                        if str(geno.split("|")[1]) != '0':
                            pos2 =str(geno.split("|")[1])

                        if str(geno.split("|")[2]) != '0':
                            pos3 =str(geno.split("|")[2])


                newString = str(pos1+"|"+pos2+"|"+pos3)
                tempDupDF.at[largest,sample]=newString
            
            
        else:
            
            largest='temp'
            largestLength=0
            for row in testDF.index:
                if len(testDF.at[row,'ALT'])>largestLength:
                    largestLength = len(testDF.at[row,'ALT'])
                    largest = row
                else:
                    continue
                    
            if largest == PALMERsinglerow:
                reverseFlag=1
            else:
                reverseFlag=0


            dedupDict[dupPosition]['Chosen']=largest
            dedupDict[dupPosition]['Duplications']=[y for y in testDF.index if y !=largest]
            
            # Merge genotyping
            for sample in sampleDict.keys():
                pos1='0'
                pos2='0'
                pos3='0'

                for row in testDF.index:
                    if 'MEI' in str(row):
                        geno = str(testDF.at[row,sample])
                        if str(geno.split("|")[0]) != '0':
                            pos1 =str(geno.split("|")[0])

                        if str(geno.split("|")[1]) != '0':
                            pos2 =str(geno.split("|")[1])

                    else:
                        geno = str(testDF.at[row,sample])
                        if str(geno.split("|")[0]) != '0':
                            pos1 =str(geno.split("|")[0])

                        if str(geno.split("|")[1]) != '0':
                            pos2 =str(geno.split("|")[1])

                        if str(geno.split("|")[2]) != '0':
                            pos3 =str(geno.split("|")[2])


                newString = str(pos1+"|"+pos2+"|"+pos3)
                tempDupDF.at[largest,sample]=newString
            
            
            if reverseFlag == 0:
                tempDupDF.at[largest,'Caller_Count']=2
                tempDupDF.at[largest,'PALMER']=1
                tempDupDF.at[largest,'PALMER_INFO']=tempDupDF.at[PALMERsinglerow,'PALMER_INFO']
                tempDupDF.at[largest,'L1ME-AID_Lab']=1
                
            else:
                singleHit = dedupDict[dupPosition]['Duplications'][0]
                
                tempDupDF.at[largest,'Caller_Count']=2
                tempDupDF.at[largest,'PALMER']=1
                tempDupDF.at[largest,'L1ME-AID_Lab']=1
                tempDupDF.at[largest,'L1ME-AID_INFO']=tempDupDF.at[singleHit,'L1ME-AID_INFO']
        

        
    else:
        print(dupPosition)
        print(set(testDF['TE_Designation']))
        print("HUH3") 
        
        
        
        
tempDupDF['MergedCalls']='NONE'
for row in tempDupDF.index:
    for element in dedupDict.keys():
        if dedupDict[element]['Chosen'] ==  row:
            tempDupDF.at[row,'MergedCalls'] = dedupDict[element]['Duplications']
        else:
            continue
            
chosenRows=[]
for x in dedupDict.keys():
    for y in dedupDict[x]['Duplications']:
        chosenRows.append(y)
        
deduplicatedDF = tempDupDF.loc[[x for x in tempDupDF.index if x not in chosenRows and x not in additionalCops]].copy()
uniquet2t2['chromLoc']='NONE'
uniquet2t2['MergedCalls']='NONE'
concatList=[uniquet2t2,deduplicatedDF]
putBackDF = pd.concat(concatList)
putBackDF.drop(columns=['chromLoc'], inplace=True)
putBackDF2 = putBackDF.reset_index().copy()
testRows=[]
for row in putBackDF2.index:
    originalID = str(putBackDF2.at[row,'ID'])
    if 'MEI' in originalID and putBackDF2.at[row,'MergedCalls']!='NONE':
        testRows.append(row)
        
        pavName = putBackDF2.at[row,'MergedCalls'][0]

        putBackDF2.at[row,'ID']=pavName
        putBackDF2.at[row,'ALT']=t2t2.at[pavName,'ALT']

        putBackDF2.at[row,'MergedCalls']='temp'
        putBackDF2.at[row,'MergedCalls']=[originalID]


        
    else:
        continue
putBackDF2.set_index("ID", inplace=True)
finalT2TDemerged = putBackDF2.copy()

chr9_141236607
{'SINE/Alu', 'Retroposon/SVA'}
HUH3


In [159]:
len(finalT2TDemerged)

13216

In [160]:
print(collections.Counter(finalT2TDemerged['TE_Designation']))

Counter({'SINE/Alu': 10689, 'LINE/L1': 1718, 'Retroposon/SVA': 803, 'HERVK': 5, 'snRNA': 1})


In [161]:
print(collections.Counter(finalT2TDemerged[finalT2TDemerged['Caller_Count']>1]['TE_Designation']))

Counter({'SINE/Alu': 9948, 'LINE/L1': 1416, 'Retroposon/SVA': 615})


In [191]:
13210/9453

1.3974399661483128

In [176]:
9948+1416+615

11979

In [178]:
11979/13216

0.9064013317191283

In [181]:
23920/2

11960.0

In [ ]:
n=9453 MEIs; Alu: 7738, L1: 1775, and SVA: 540

In [188]:
12994/9453

1.3745900772241617

In [162]:
print(collections.Counter(finalhg38Demerged['TE_Designation']))

Counter({'SINE/Alu': 10522, 'LINE/L1': 1702, 'Retroposon/SVA': 770, 'HERVK': 6, 'snRNA': 1})


In [182]:
len(finalhg38Demerged)

13001

In [163]:
print(collections.Counter(finalhg38Demerged[finalhg38Demerged['Caller_Count']>1]['TE_Designation']))

Counter({'SINE/Alu': 9932, 'LINE/L1': 1404, 'Retroposon/SVA': 605})


In [177]:
9932+1404+605

11941

In [179]:
11941/13001

0.9184678101684486

In [167]:
#finalhg38Demerged.to_csv("/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/GRCh38_hgsvc3_PublicationSet_Final.csv")
#finalT2TDemerged.to_csv("/home/mark/Desktop/MEI_Group/HGSVC3/Merged_Callsets/Manuscript/t2tCHM13_hgsvc3_PublicationSet_Final.csv")